###  Оптимальная шкала средних ставок подоходного налога:

 

 \begin{equation*}
	y_{opt}(x) = 
	 \begin{cases}
	  	y_- ,   &0\leq x<x_-,  \\
 	  	1 - \left ( \frac{x_-}{x} \right )^\sigma , &x_-\leq x<x_0,  \\
        1 -(1-y_+) \left ( \frac{x_+}{x} \right )^\delta , &x_0\leq x \leq x_+,  \\
        y_+ , &x>x_+;  \\
	 \end{cases}
 \end{equation*}



#### Суммарные налоговые поступления

$$T(y,f) = \int_{x_-}^{\infty} y(x) dI(x)$$ где I(x) - суммарный доход тех, чей доход <x

#### Суммарный доход тех, чей доход < x

$$I(x) = m\int_{0}^{x} f(t)t  dt$$где f(t) - плотность распределения доходов 
m - 

In [1]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
import scipy.stats as st
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

### Калькулятор оптимальной шкалы ставок и налоговых поступлений:

In [2]:
def parameters(x_,kx_zero,kx_plus, y_plus, y_zero, nu, sgm, empl):
    print(15*'.','Итоговые значения:',15*'.')
    print('Прожиточный минимум - ' , str(x_)+',', 'за год - ', 12*x_)
    print('Нижняя граница доходов среднего класса:' ,12*kx_zero*x_)
    print('Размер дохода налогоплательщица, начиная с которого доход взимается по максимальной средней ставке у+:' ,12*kx_plus*x_)
    print('Максимальная ставка подоходного налога:', y_plus,'%')
    print(50*'.')
    left_border = 1-(1-y_plus/100)**((np.log(12*x_) - np.log(12*kx_zero*x_))/(np.log(12*x_) - np.log(12*kx_plus*x_)))
    #print('{0:.4f}'.format(left_border), ' < Yo < ', np.min([y_plus/100, 1-1/kx_zero]))
    print( int(left_border*100),'%', ' < Y0 < ', int(100*np.min([y_plus/100, 1-1/kx_zero])),'%')
    
    yos = np.arange(int(left_border*100)+1,int(100*np.min([y_plus/100, 1-1/kx_zero])))
    table = pd.DataFrame([str(yos[i])+'%' for i in range(len(yos))], columns=['Y0'])
    
    table.loc[:, 'delta'] = [(np.log((1-yos[i]/100)/(1-y_plus/100))/np.log(kx_plus/kx_zero)) for i in range(len(yos))]
    table.loc[:, 'sigma'] = [np.log(1-yos[i]/100)/np.log(1/kx_zero) for i in range(len(yos))]
    
    print('Заданная граница макс дохода =',12*kx_plus*x_*1.05)
    print(50*'.')
    s=[0 for i in range(len(yos))]    
    for i in range(len(yos)):
        x1 = np.arange(0, 12*x_, 1)
        x2 = np.arange(12*x_, 12*kx_zero*x_, 1)
        x3 = np.arange(12*kx_zero*x_, 12*kx_plus*x_, 1)
        x4 = np.arange(12*kx_plus*x_, 9999999, 1)

        y1 = 0*x1
        y2 = 1-(12*x_/x2)**table['sigma'][i]
        y3 = 1-(1-y_plus/100)*((12*kx_plus*x_/x3)**table['delta'][i])
        y4 = np.full(len(x4),y_plus/100)
            
        T1 = y1*st.lognorm.pdf(x1, 1, loc=nu, scale=sgm)*empl*x1
        T2 = y2*st.lognorm.pdf(x2, 1, loc=nu, scale=sgm)*empl*x2
        T3 = y3*st.lognorm.pdf(x3, 1, loc=nu, scale=sgm)*empl*x3
        T4 = y4*st.lognorm.pdf(x4, 1, loc=nu, scale=sgm)*empl*x4

        xs = [x1,x2,x3,x4]
        ys = [T1,T2,T3,T4]
 
        for x,y in zip(xs,ys):
            s[i] += simps(y, x)
        
    table.loc[:, 'Налоговые пост.'] = s
    table['Поступления, млрд. руб.'] = (table['Налоговые пост.'].astype(float)/1000000000).astype(str) + ' млрд. руб.'
    
    print(table)
    #table.to_excel(pd.ExcelWriter('Расчеты.xlsx', engine='xlsxwriter'),)
    print(50*'.') 
    cols =[(0/255, 183/255, 213/255),(3/255, 209/255, 175/255),(230/255, 158/255, 0/255),(255/255, 112/255, 67/255)]

    plt.rcParams['figure.figsize'] = [15, 6]
    plt.rcParams['figure.edgecolor'] = 'blue'
    plt.rcParams['axes.facecolor'] = 'white'
    plt.rcParams['axes.edgecolor'] = 'white'
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['font.size'] = 17
    plt.rcParams['text.color'] = (43/255, 43/255, 43/255)
    plt.rcParams['grid.color'] = (231/255, 243/255, 247/255)
    plt.rcParams['grid.linestyle'] = '-'
    plt.rcParams['axes.labelcolor'] = (43/255, 43/255, 43/255)
    plt.rcParams['xtick.color'] = (43/255, 43/255, 43/255)
    plt.rcParams['ytick.color'] = (43/255, 43/255, 43/255)
    plt.ylabel('Y', size=17)
    plt.xlabel('\nДоход', size=22)

    for y_zero in yos:
        #Прорисовка графика
        x1 = np.arange(0,12*x_,1)
        x2 = np.arange(12*x_,12*kx_zero*x_,1)
        x3 = np.arange(12*kx_zero*x_,12*kx_plus*x_,1)
        x4 = np.arange(12*kx_plus*x_, 12*kx_plus*x_*1.05,1)
        ########
        y1 = 0*x1
        y2 =1-(12*x_/x2)**(np.log(1-y_zero/100)/np.log(1/kx_zero))
        y3 = 1-(1-y_plus/100)*((12*kx_plus*x_/x3)**(np.log((1-y_zero/100)/(1-y_plus/100))/np.log(kx_plus/kx_zero)))
        y4 = np.full(len(x4),y_plus/100)
        ########
        xs = [x1,x2,x3,x4]
        ys = [y1,y2,y3,y4] 
        sm = 0
        for i,j,c in zip(xs,ys,cols):
            plt.plot(i,j, color=c)
            
widgets.interact(parameters, 

                 x_ = widgets.IntText(
                    value=9828,
                    description='X_:',
                    disabled=False),
                 kx_zero = widgets.IntText(
                    value=5,
                    description='Коэф. Xо:',
                    disabled=False),
                 kx_plus = widgets.IntText(
                    value=20,
                    description='Коэф. X+:',
                    disabled=False),
                 y_plus = widgets.IntText(
                    value=25,
                    description='Y+,%:',
                    disabled=False),
                 y_zero = widgets.IntText(
                    value=15,
                    description='Выбран Yo,%:',
                    disabled=False),
                 nu = widgets.FloatText(
                    value=29376.5,
                    description='mean wage:',
                    disabled=False),
                 sgm = widgets.FloatText(
                    value=21606.43417,
                    description='std wage:',
                    disabled=False),
                 empl = widgets.FloatText(
                    value=72392628,
                    description='num of empl',
                    disabled=False),
                )

interactive(children=(IntText(value=9828, description='X_:'), IntText(value=5, description='Коэф. Xо:'), IntTe…

<function __main__.parameters>